# Tachyonic condensation · Cylinder modes walkthrough

A quick look at `src/tachyonic_condensation/cylinder_modes.py`. We check SciPy availability, compute a few κ-roots when possible, and inspect the mode metadata.


In [ ]:
import sys
from pathlib import Path


def _ensure_repo_on_path():
    cwd = Path.cwd().resolve()
    candidates = [cwd, cwd.parent, cwd.parent.parent]
    for candidate in candidates:
        utils_py = candidate / 'notebooks' / 'utils.py'
        if utils_py.exists():
            sys.path.insert(0, str(candidate))
            return candidate
    return cwd

_ensure_repo_on_path()

from notebooks.utils import ensure_repo_path, preview_json

repo_root = ensure_repo_path()
print(f"Using repo root: {repo_root}")



In [ ]:
try:
    from src.tachyonic_condensation import cylinder_modes
except ModuleNotFoundError:
    from notebooks.utils import ensure_repo_path as _vdm_ensure_repo_path
    _vdm_ensure_repo_path()
    import importlib
    importlib.invalidate_caches()
    from src.tachyonic_condensation import cylinder_modes

modes = []
if not getattr(cylinder_modes, '_HAVE_SCIPY', False):
    print('SciPy is not available; install scipy to solve for cylinder modes.')
else:
    modes = cylinder_modes.compute_kappas(R=3.0, mu=1.0, c=1.0, ell_max=2, num_brackets=64, tol=1e-6)
    snippet = [{k: float(v) if isinstance(v, (int, float)) else v for k, v in mode.items()} for mode in modes[:3]]
    print(preview_json({"mode_count": len(modes), "first_modes": snippet}))


In [ ]:
if getattr(cylinder_modes, '_HAVE_SCIPY', False):
    import matplotlib.pyplot as plt
    kappas = [float(mode['kappa']) for mode in modes]
    ells = [int(mode['ell']) for mode in modes]
    plt.figure(figsize=(5, 3))
    plt.stem(ells, kappas, use_line_collection=True)
    plt.xlabel('ell')
    plt.ylabel('kappa')
    plt.title('Cylinder mode spectrum (demo)')
    plt.tight_layout()
    plt.show()


## Notebook checklist

* A SciPy install is required because the solver relies on Bessel functions and root finding.
* Adjust `ell_max`, `R`, or `mu` to explore different tube radii and spectra.
* Reuse the returned list of dicts as input to the condensation module or other diagnostics.
